# Import

In [1]:
!python -m spacy download pt_core_news_sm

2023-12-26 22:18:43.508249: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-26 22:18:43.508312: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-26 22:18:43.510477: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-26 22:18:43.527246: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-26 22:18:50.369960: W tensorflow/compiler/tf2

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import spacy
import numpy as np
import pandas as pd
from gensim.models import KeyedVectors
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import pickle


path_treino = "/content/drive/MyDrive/w2v-Alura/treino_caelum.csv"
path_teste = "/content/drive/MyDrive/w2v-Alura/teste_caelum.csv"

In [4]:
w2v_modelo_cbow = KeyedVectors.load_word2vec_format("/content/drive/MyDrive/w2v-Alura/modelo_cbow.txt")
w2v_modelo_sg = KeyedVectors.load_word2vec_format("/content/drive/MyDrive/w2v-Alura/modelo_skipgram.txt")

artigo_treino = pd.read_csv(path_treino)
artigo_teste = pd.read_csv(path_teste)

# Funções

In [5]:
nlp = spacy.load("pt_core_news_sm", disable=["paser", "ner", "tagger", "textcat"])

def tokenizador(texto):

    doc = nlp(texto)
    tokens_validos = []
    for token in doc:
        e_valido = not token.is_stop and token.is_alpha
        if e_valido:
            tokens_validos.append(token.text.lower())

    return  tokens_validos

texto = "Rio de Janeiro 1231231 ***** @#$ é uma cidade maravilhosa!"
tokens = tokenizador(texto)
print(tokens)

['rio', 'janeiro', 'cidade', 'maravilhosa']


In [8]:
def combinacao_de_vetores_por_soma(palavras, modelo):

    vetor_resultante = np.zeros((1,300))

    for pn in palavras:
        try:
            vetor_resultante += modelo.get_vector(pn)
        except KeyError:
            # Bola pra frente
            pass
    return vetor_resultante

vetor_texto = combinacao_de_vetores_por_soma(tokens, w2v_modelo_cbow)
print(vetor_texto.shape)
print(vetor_texto)

(1, 300)
[[ 3.90695944e-01  6.59937561e-01  2.52442360e-01 -8.25221434e-01
   4.64935843e-01  1.59929344e+00 -1.86455211e+00  1.86138928e+00
  -4.43210341e-02 -6.92379199e-01  8.27004656e-01  8.77119452e-01
  -1.50367618e+00 -6.05322719e-01  4.44926769e-01 -2.23134655e+00
   8.92919697e-01 -1.80792961e+00  6.30468678e-01  9.55476880e-01
   1.02653295e-01 -9.38651994e-01 -3.02177519e-02  1.14091158e-01
   5.70079714e-01 -2.46944949e+00  1.34978160e+00  3.61830581e-01
   7.43960261e-01  9.37703934e-01 -4.50169653e-01 -2.90026897e+00
  -1.51684135e-02  2.31903291e+00  1.07263330e+00 -6.85373724e-01
  -2.26547894e+00 -1.66016987e+00 -6.62344038e-01 -8.24440565e-01
  -6.96291104e-01  9.27211896e-01  6.16904132e-01 -9.40768778e-01
  -2.12425026e+00 -9.40065175e-01  2.97240478e+00 -1.00502357e+00
   1.64995905e-01 -4.57552344e-01 -5.03328237e-01 -1.60699466e+00
  -6.62588410e-01 -2.14379352e+00 -9.53487754e-02  6.21271312e-01
  -1.95610836e-01 -1.63215154e+00  8.13400842e-01  6.52666152e-01
 

In [9]:
def matriz_vetores(textos, modelo):
    x = len(textos)
    y = 300
    matriz = np.zeros((x,y))

    for i in range(x):
        palavras = tokenizador(textos.iloc[i])
        matriz[i] = combinacao_de_vetores_por_soma(palavras, modelo)

    return matriz

matriz_vetores_treino_cbow = matriz_vetores(artigo_treino.title, w2v_modelo_cbow)
matriz_vetores_teste_cbow = matriz_vetores(artigo_teste.title, w2v_modelo_cbow)

print(matriz_vetores_treino_cbow.shape)
print(matriz_vetores_teste_cbow.shape)

(90000, 300)
(20513, 300)


# Classificação

In [10]:
def classificador(modelo, x_treino, y_treino, x_teste, y_teste):
    logreg = LogisticRegression(max_iter = 800)
    logreg.fit(x_treino, y_treino)

    categ_prevista = logreg.predict(x_teste)

    resultados = classification_report(y_teste, categ_prevista)
    print(resultados)

    return logreg

              precision    recall  f1-score   support

     colunas       0.80      0.71      0.75      6103
   cotidiano       0.64      0.80      0.71      1698
     esporte       0.93      0.86      0.89      4663
   ilustrada       0.13      0.83      0.22       131
     mercado       0.84      0.78      0.81      5867
       mundo       0.74      0.84      0.79      2051

    accuracy                           0.78     20513
   macro avg       0.68      0.80      0.69     20513
weighted avg       0.81      0.78      0.80     20513



In [11]:
logreg_cbow = classificador(w2v_modelo_cbow,
                        matriz_vetores_treino_cbow,
                         artigo_treino.category,
                        matriz_vetores_teste_cbow,
                        artigo_teste.category)

              precision    recall  f1-score   support

     colunas       0.80      0.71      0.75      6103
   cotidiano       0.64      0.80      0.71      1698
     esporte       0.93      0.86      0.89      4663
   ilustrada       0.13      0.83      0.22       131
     mercado       0.84      0.78      0.81      5867
       mundo       0.74      0.84      0.79      2051

    accuracy                           0.78     20513
   macro avg       0.68      0.80      0.69     20513
weighted avg       0.81      0.78      0.80     20513



In [12]:
matriz_vetores_treino_sg = matriz_vetores(artigo_treino.title, w2v_modelo_sg)
matriz_vetores_teste_sg = matriz_vetores(artigo_teste.title, w2v_modelo_sg)

In [13]:
logreg_sg = classificador(w2v_modelo_sg,
                        matriz_vetores_treino_sg,
                        artigo_treino.category,
                        matriz_vetores_teste_sg,
                        artigo_teste.category)

              precision    recall  f1-score   support

     colunas       0.81      0.72      0.76      6103
   cotidiano       0.64      0.80      0.71      1698
     esporte       0.94      0.87      0.90      4663
   ilustrada       0.14      0.87      0.24       131
     mercado       0.84      0.79      0.81      5867
       mundo       0.75      0.84      0.79      2051

    accuracy                           0.79     20513
   macro avg       0.69      0.82      0.70     20513
weighted avg       0.82      0.79      0.80     20513



In [14]:
with open("/content/drive/My Drive/w2v-Alura/rl_cbow.pkl", "wb") as f:
    # quero escrever isso
    pickle.dump(logreg_cbow, f)

In [15]:
with open("/content/drive/My Drive/w2v-Alura/rl_sg.pkl", "wb") as f:
    pickle.dump(logreg_sg, f)